# Kaggle
## Competition NFL Big Data Bowl

In [24]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [43]:
# Carregando os pacotes
import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# Statistic lib
from scipy import stats
from scipy.stats import skew, norm, kurtosis

# Sklearn lib
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder

# Models
from xgboost import XGBRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from mlxtend.regressor import StackingCVRegressor
from lightgbm import LGBMRegressor
import lightgbm as lgb
import xgboost as XGB
from sklearn.cluster import KMeans
import tqdm

# Misc lib
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, cross_val_score
from sklearn.model_selection import RandomizedSearchCV
from functools import partial
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from IPython.display import Image

# Utils
import pandasql as ps
import re 
import math, string, os
import datetime

# Options
import warnings
warnings.filterwarnings('ignore')
pd.options.display.max_seq_items = 8000
pd.options.display.max_rows = 8000
pd.set_option('display.max_columns', None)
import gc
gc.enable()

In [26]:
# Carregando os dados de treino
train = pd.read_csv('../data/train.csv', low_memory=False)
#train = pd.read_csv('/kaggle/input/nfl-big-data-bowl-2020/train.csv', low_memory=False)
print ("Data is ready !!")

Data is ready !!


# Criando as funções auxiliares de limpeza e conversao

In [66]:
# Funcao para tratar os dados missing de cada variavel
def fill_na(data):
    data['WindDirection'].fillna('unknown',inplace=True)
    data['OffenseFormation'].fillna('unknown',inplace=True)
    data['StadiumType'].fillna('unknown',inplace=True)
    data['GameWeather'].fillna('unknown',inplace=True)
    data['FieldPosition'].fillna('NA',inplace=True)
    
    data['Temperature'].fillna(data['Temperature'].mean(), inplace=True)
    data['Humidity'].fillna(data['Humidity'].mean(), inplace=True)
    data['DefendersInTheBox'].fillna(math.ceil(data['DefendersInTheBox'].mean()),inplace=True)
    return data
    
# Funcao para agrupar as descricoes dos tipos de estadio
def agrupar_tipo_estadio(StadiumType):
    outdoor       = ['Outdoor', 'Outdoors', 'Cloudy', 'Heinz Field', 'Outdor', 'Ourdoor', 'Outside', 'Outddors', 'Outdoor Retr Roof-Open', 'Oudoor', 'Bowl']
    indoor_closed = ['Indoors', 'Indoor', 'Indoor, Roof Closed', 'Indoor, Roof Closed', 'Retractable Roof', 'Retr. Roof-Closed', 'Retr. Roof - Closed', 'Retr. Roof Closed']
    indoor_open   = ['Indoor, Open Roof', 'Open', 'Retr. Roof-Open', 'Retr. Roof - Open']
    dome_closed   = ['Dome', 'Domed, closed', 'Closed Dome', 'Domed', 'Dome, closed']
    dome_open     = ['Domed, Open', 'Domed, open']
    
    if StadiumType in outdoor:
        return 'outdoor'
    elif StadiumType in indoor_closed:
        return 'indoor_closed'
    elif StadiumType in indoor_open:
        return 'indoor_open'
    elif StadiumType in dome_closed:
        return 'dome_closed'
    elif StadiumType in dome_open:
        return 'dome_open'
    else:
        return 'unknown' # se for n/a
    
# Funcao para agrupar as descricoes dos estadios
def agrupar_estadio(Stadium):

    if Stadium == 'Broncos Stadium at Mile High':
        return 'Broncos Stadium At Mile High'
    
    elif Stadium in ('CenturyField', 'CenturyLink'):
        return 'CenturyLink Field'
    
    elif Stadium == 'EverBank Field':
        return 'Everbank Field'
    
    elif Stadium in ('FirstEnergy', 'FirstEnergy Stadium', 'FirstEnergyStadium'):
        return 'First Energy Stadium'
   
    elif Stadium == 'Lambeau field':
        return 'Lambeau Field'

    elif Stadium == 'Los Angeles Memorial Coliesum':
        return 'Los Angeles Memorial Coliseum'
    
    elif Stadium in ('M & T Bank Stadium', 'M&T Stadium'):
        return 'M&T Bank Stadium'

    elif Stadium in ('Mercedes-Benz Dome', 'Mercedes-Benz Superdome'):
        return 'Mercedes-Benz SuperDome'
    
    elif Stadium in ('MetLife Stadium', 'Metlife Stadium', 'MetLife'):
        return 'MetLife Stadium' 
    
    elif Stadium == 'NRG':
        return 'NRG Stadium' 

    elif Stadium == 'Oakland-Alameda County Coliseum':
        return 'Oakland Alameda-County Coliseum' 
    
    elif Stadium == 'Paul Brown Stdium':
        return 'Paul Brown Stadium' 

    elif Stadium == 'Twickenham':
        return 'Twickenham Stadium' 
    
    else:
        return Stadium
    
# Funcao para agrupar a localizacao do estadio e do jogo
def agrupar_local(Location):

    if Location == "Arlington, Texas":
        return "Arlington, TX"
    elif Location in ("Baltimore, Maryland","Baltimore, Md."):
        return "Baltimore, MD"
    elif Location == "Charlotte, North Carolina":
        return "Charlotte, NC"
    elif Location == "Chicago. IL":
        return "Chicago, IL"
    elif Location == "Cincinnati, Ohio":
        return "Cincinnati, OH"
    elif Location in ("Cleveland","Cleveland Ohio","Cleveland, Ohio","Cleveland,Ohio"):
        return "Cleveland, OH"
    elif Location == "Detroit":
        return "Detroit, MI"
    elif Location == "E. Rutherford, NJ" or Location == "East Rutherford, N.J.":
        return "East Rutherford, NJ"
    elif Location == "Foxborough, Ma":
        return "Foxborough, MA"
    elif Location == "Houston, Texas":
        return "Houston, TX"
    elif Location in ("Jacksonville Florida","Jacksonville, Fl","Jacksonville, Florida"):
        return "Jacksonville, FL"
    elif Location == "London":
        return "London, England"
    elif Location == "Los Angeles, Calif.":
        return "Los Angeles, CA"
    elif Location == "Miami Gardens, Fla.":
        return "Miami Gardens, FLA"
    elif Location in ("New Orleans","New Orleans, La."):
        return "New Orleans, LA"
    elif Location == "Orchard Park NY":
        return "Orchard Park, NY"
    elif Location == "Philadelphia, Pa.":
        return "Philadelphia, PA"
    elif Location == "Pittsburgh":
        return "Pittsburgh, PA"
    elif Location == "Seattle":
        return "Seattle, WA"
    else:
        return Location
    
# Funcao para agrupar o gramado do estadio
def agrupar_gramado(Turf):
    if Turf == 'Artifical':
        return 'Artificial'
    
    elif Turf in ('FieldTurf', 'Field turf'):
        return 'Field Turf'

    elif Turf in ('FieldTurf360', 'FieldTurf 360'):
        return 'Field Turf 360'

    elif Turf in ('Natural', 'Natural grass', 'Naturall Grass', 'grass', 'natural grass', 'SISGrass', 'Natural Grass'):
        return "Grass"

    elif Turf == "UBU Sports Speed S5-M":
        return "UBU Speed Series-S5-M"

    else:
        return Turf

# Funcao para agrupar os dados de direcao do vento
def agrupa_wind_direction(WindDirection):
    wd = str(WindDirection).upper()
    
    if wd == 'N' or 'FROM N' in wd:
        return 'north'
    if wd == 'S' or 'FROM S' in wd:
        return 'south'
    if wd == 'W' or 'FROM W' in wd:
        return 'west'
    if wd == 'E' or 'FROM E' in wd:
        return 'east'
    
    if 'FROM SW' in wd or 'FROM SSW' in wd or 'FROM WSW' in wd:
        return 'south west'
    if 'FROM SE' in wd or 'FROM SSE' in wd or 'FROM ESE' in wd:
        return 'south east'
    if 'FROM NW' in wd or 'FROM NNW' in wd or 'FROM WNW' in wd:
        return 'north west'
    if 'FROM NE' in wd or 'FROM NNE' in wd or 'FROM ENE' in wd:
        return 'north east'
    
    if 'NW' in wd or 'NORTHWEST' in wd:
        return 'north west'
    if 'NE' in wd or 'NORTH EAST' in wd:
        return 'north east'
    if 'SW' in wd or 'SOUTHWEST' in wd:
        return 'south west'
    if 'SE' in wd or 'SOUTHEAST' in wd:
        return 'south east'

    return 'unknown'

# Funcao para agrupar as descricoes de clima
def agrupar_clima(GameWeather):
    chuva   = ['Rainy', 'Rain Chance 40%', 'Showers',
               'Cloudy with periods of rain, thunder possible. Winds shifting to WNW, 10-20 mph.',
               'Scattered Showers', 'Cloudy, Rain', 'Rain shower', 'Light Rain', 'Rain']
    nublado = ['Cloudy, light snow accumulating 1-3"', 'Party Cloudy', 'Cloudy, chance of rain',
               'Coudy', 'Cloudy, 50% change of rain', 'Rain likely, temps in low 40s.',
               'Cloudy and cold', 'Cloudy, fog started developing in 2nd quarter',
               'Partly Clouidy', '30% Chance of Rain', 'Mostly Coudy', 'Cloudy and Cool',
               'cloudy', 'Partly cloudy', 'Overcast', 'Hazy', 'Mostly cloudy', 'Mostly Cloudy',
               'Partly Cloudy', 'Cloudy']
    limpo   = ['Partly clear', 'Sunny and clear', 'Sun & clouds', 'Clear and Sunny',
               'Sunny and cold', 'Sunny Skies', 'Clear and Cool', 'Clear and sunny',
               'Sunny, highs to upper 80s', 'Mostly Sunny Skies', 'Cold',
               'Clear and warm', 'Sunny and warm', 'Clear and cold', 'Mostly sunny',
               'T: 51; H: 55; W: NW 10 mph', 'Clear Skies', 'Clear skies', 'Partly sunny',
               'Fair', 'Partly Sunny', 'Mostly Sunny', 'Clear', 'Sunny']
    neve    = ['Heavy lake effect snow', 'Snow']
    none    = ['N/A Indoor', 'Indoors', 'Indoor', 'N/A (Indoors)', 'Controlled Climate']

    
    if GameWeather in chuva:
        return 'chuva'
    elif GameWeather in nublado:
        return 'nublado'
    elif GameWeather in limpo:
        return 'limpo'
    elif GameWeather in neve:
        return 'neve'
    elif GameWeather in none:
        return 'none'
    else:
        return 'none' # se for n/a
    
# Funcao para converter a velocidade do vento
def convert_wind_speed(WindSpeed):
    ws = str(WindSpeed)

    if ws.isdigit():
        return int(ws)

    if '-' in ws:
        return int(ws.split('-')[0])

    if ws.split(' ')[0].isdigit():
        return int(ws.split(' ')[0])

    if 'mph' in ws.lower():
        return int(ws.lower().split('mph')[0])
    else:
        return 0
    
    
# Funcao para converter altura de feet-inches para centimetros
def convert_to_cm(ft_in):
    h_ft   = int(ft_in.split('-')[0])
    h_inch = int(ft_in.split('-')[1])
    h_inch += h_ft * 12
    h_cm = round(h_inch * 2.54, 1)
    #print("Your height is : %d cm." % h_cm)   
    
    return h_cm

# Funcao para converter peso em lbs para kg
def convert_to_kg(lbs):
    kg = lbs * 0.45359237
    #print("The weight is", kg, "in kilograms")
    
    return kg

# Funcao para converter temperatura Fahrenheit para Celsius
def convert_to_celsius(fah):
    celsius = (fah - 32) * 5.0/9.0
    #print("Temperature:", fah, "Fahrenheit = ", celsius, " C")
    return celsius
    
    
# Funcao para converter uma string horario em segundos
def str_to_seconds(time):
    time = time.split(':')
    sec = int(time[0])*60 + int(time[1]) + int(time[2])/60
    return sec


def _kurtosis(x):
    return kurtosis(x)

def CPT5(x):
    den = len(x)*np.exp(np.std(x))
    return sum(np.exp(x))/den

def skewness(x):
    return skew(x)

def SSC(x):
    x = np.array(x)
    x = np.append(x[-1], x)
    x = np.append(x,x[1])
    xn = x[1:len(x)-1]
    xn_i2 = x[2:len(x)]    # xn+1 
    xn_i1 = x[0:len(x)-2]  # xn-1
    ans = np.heaviside((xn-xn_i1)*(xn-xn_i2),0)
    return sum(ans[1:]) 

def wave_length(x):
    x = np.array(x)
    x = np.append(x[-1], x)
    x = np.append(x,x[1])
    xn = x[1:len(x)-1]
    xn_i2 = x[2:len(x)]    # xn+1 
    return sum(abs(xn_i2-xn))
    
def norm_entropy(x):
    tresh = 3
    return sum(np.power(abs(x),tresh))

def SRAV(x):    
    SRA = sum(np.sqrt(abs(x)))
    return np.power(SRA/len(x),2)

def mean_abs(x):
    return sum(abs(x))/len(x)

def zero_crossing(x):
    x = np.array(x)
    x = np.append(x[-1], x)
    x = np.append(x,x[1])
    xn = x[1:len(x)-1]
    xn_i2 = x[2:len(x)]    # xn+1
    return sum(np.heaviside(-xn*xn_i2,0))

def mean_change_of_abs_change(x):
    return np.mean(np.diff(np.abs(np.diff(x))))

# Funcao para a metrica de validacao do modelo
def funcao_crps(labels,predictions) :
    y_pred = np.zeros((len(labels),199))
    y_ans = np.zeros((len(labels),199))
    j = np.array(range(199))
    for i,(p,t) in enumerate(zip(np.round(scaler.inverse_transform(predictions)),labels)) :
        k2 = j[j>=p-10]
        y_pred[i][k2]=(k2+10-p)*0.05
        k1 = j[j>=p+10]
        y_pred[i][k1]= 1.0
        k3 = j[j>=t]
        y_ans[i][k3]= 1.0
                           
    return 'CRPS: ', np.sum((y_pred-y_ans)**2)/(199*y_pred.shape[0]), False

# Feature Engineering

In [28]:
# Tirando uma copia para backup
train_df = train.copy()

In [29]:
# Preenche os valores missing
train_df = fill_na(train_df)

In [30]:
# Funcao para realizar feature engineering no dataset treino
def feature_basicas(df): 
    
    # Converter as features de data e extrair dia, mes, ano, hora, minuto, segundo
    df['PlayerBirthDate'] = df['PlayerBirthDate'].apply(lambda x: datetime.datetime.strptime(x, "%m/%d/%Y"))
    df['PlayerBirthDate_day'] = df['PlayerBirthDate'].dt.day.astype(int)
    df['PlayerBirthDate_month'] = df['PlayerBirthDate'].dt.month.astype(int)
    df['PlayerBirthDate_year'] = df['PlayerBirthDate'].dt.year.astype(int)

    df['TimeSnap'] = df['TimeSnap'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%dT%H:%M:%S.%fZ"))
    df['TimeSnap_min'] = df['TimeSnap'].dt.minute.astype(int)
    df['TimeSnap_seg'] = df['TimeSnap'].dt.second.astype(int)
    
    df['TimeHandoff'] = df['TimeHandoff'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%dT%H:%M:%S.%fZ"))
    df['TimeHandoff_min'] = df['TimeHandoff'].dt.minute.astype(int)
    df['TimeHandoff_seg'] = df['TimeHandoff'].dt.second.astype(int)
    
    # Conversao de algumas features
    df['PlayerHeight']  = df['PlayerHeight'].apply(convert_to_cm)
    df['PlayerWeight']  = df['PlayerWeight'].apply(convert_to_kg)
    df['Temperature']   = df['Temperature'].apply(convert_to_celsius)
    df['StadiumType']   = df['StadiumType'].apply(agrupar_tipo_estadio)
    df['Stadium']       = df['Stadium'].apply(agrupar_estadio)
    df['Location']      = df['Location'].apply(agrupar_local)
    df['Turf']          = df['Turf'].apply(agrupar_gramado)
    df['WindDirection'] = df['WindDirection'].apply(agrupa_wind_direction)
    df['WindSpeed']     = df['WindSpeed'].apply(convert_wind_speed)
    df['GameWeather']   = df['GameWeather'].apply(agrupar_clima)


    # Corrigindo a feature Stadium
    df.loc[df['Stadium'] == 'MetLife Stadium', 'StadiumType'] = 'outdoor'
    df.loc[df['Stadium'] == 'StubHub Center', 'StadiumType'] = 'outdoor'    
    
    # Nova feature com a diferença entre o tempo de lançamento da bola até quando o jogador captura
    df['TimeDifer'] = df.apply(lambda row: (row['TimeHandoff'] - row['TimeSnap']).total_seconds(), axis=1)            
         
    # Nova feature para indicar se é o jogador que esta realizando a jogada (corredor)
    df['IsRusher'] = df['NflId'] == df['NflIdRusher']
    
    # Novas features com base no horario do jogo
    df['Morning']   = df['GameClock'].apply(lambda x : 1 if (int(x[0:2]) >=0 and int(x[0:2]) <12) else 0)
    df['Afternoon'] = df['GameClock'].apply(lambda x : 1 if (int(x[0:2]) <18 and int(x[0:2]) >=12) else 0)
    df['Evening']   = df['GameClock'].apply(lambda x : 1 if (int(x[0:2]) >= 18 and int(x[0:2]) < 24) else 0)
    df['GameClock'] = df['GameClock'].apply(str_to_seconds) 
    
    # Criando novas features com dados de Distance, YardLine e DefendersInTheBox
    df['seconds_need_to_first_down'] = (df['Distance']*0.9144)/df['Dis']
    df['seconds_need_to_YardsLine'] = (df['YardLine']*0.9144)/df['Dis']    
    df['DefendersInTheBox_vs_Distance'] = df['DefendersInTheBox'] / df['Distance']

    # Ordenacao do dataset e renovando o index
    df = df.sort_values(by=['PlayId', 'Team', 'IsRusher']).reset_index()
    
    # Removendo colunas que não serão utilizadas
    #df = df.drop(['index','Yards','GameId','PlayId','NflId', 'DisplayName','NflIdRusher', 'TimeHandoff', 'TimeSnap', 'PlayerBirthDate'], axis=1)
    
    # Atribuindo media para os demais dados missing
    df_median = df.median()
    df.fillna(df_median, inplace=True)
    
    return df

In [31]:
# Criando um novo dataset aplicando Feature Engineering
train_df = feature_basicas(train_df)

In [32]:
train_df.shape

(509762, 65)

In [33]:
train_df.head()

,index,GameId,PlayId,Team,X,Y,S,A,Dis,Orientation,Dir,NflId,DisplayName,JerseyNumber,Season,YardLine,Quarter,GameClock,PossessionTeam,Down,Distance,FieldPosition,HomeScoreBeforePlay,VisitorScoreBeforePlay,NflIdRusher,OffenseFormation,OffensePersonnel,DefendersInTheBox,DefensePersonnel,PlayDirection,TimeHandoff,TimeSnap,Yards,PlayerHeight,PlayerWeight,PlayerBirthDate,PlayerCollegeName,Position,HomeTeamAbbr,VisitorTeamAbbr,Week,Stadium,Location,StadiumType,Turf,GameWeather,Temperature,Humidity,WindSpeed,WindDirection,PlayerBirthDate_day,PlayerBirthDate_month,PlayerBirthDate_year,TimeSnap_min,TimeSnap_seg,TimeHandoff_min,TimeHandoff_seg,TimeDifer,IsRusher,Morning,Afternoon,Evening,seconds_need_to_first_down,seconds_need_to_YardsLine,DefendersInTheBox_vs_Distance
0,0,2017090700,20170907000118,away,73.91,34.84,1.69,1.13,0.40,81.99,177.18,496723,Eric Berry,29,2017,35,1,854.0,NE,3,2,NE,0,0,2543773,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"2 DL, 3 LB, 6 DB",left,2017-09-08 00:44:06,2017-09-08 00:44:05,8,182.9,96.161582,1988-12-29,Tennessee,SS,NE,KC,1,Gillette Stadium,"Foxborough, MA",outdoor,Field Turf,limpo,17.222222,77.0,8,south west,29,12,1988,44,5,44,6,1.0,False,0,1,0,4.572000,80.01000,3.0
1,1,2017090700,20170907000118,away,74.67,32.64,0.42,1.35,0.01,27.61,198.70,2495116,Allen Bailey,97,2017,35,1,854.0,NE,3,2,NE,0,0,2543773,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"2 DL, 3 LB, 6 DB",left,2017-09-08 00:44:06,2017-09-08 00:44:05,8,190.5,130.634603,1989-03-25,Miami,DE,NE,KC,1,Gillette Stadium,"Foxborough, MA",outdoor,Field Turf,limpo,17.222222,77.0,8,south west,25,3,1989,44,5,44,6,1.0,False,0,1,0,182.880000,3200.40000,3.0
2,2,2017090700,20170907000118,away,74.00,33.20,1.22,0.59,0.31,3.01,202.73,2495493,Justin Houston,50,2017,35,1,854.0,NE,3,2,NE,0,0,2543773,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"2 DL, 3 LB, 6 DB",left,2017-09-08 00:44:06,2017-09-08 00:44:05,8,190.5,122.469940,1989-01-21,Georgia,DE,NE,KC,1,Gillette Stadium,"Foxborough, MA",outdoor,Field Turf,limpo,17.222222,77.0,8,south west,21,1,1989,44,5,44,6,1.0,False,0,1,0,5.899355,103.23871,3.0
3,3,2017090700,20170907000118,away,71.46,27.70,0.42,0.54,0.02,359.77,105.64,2506353,Derrick Johnson,56,2017,35,1,854.0,NE,3,2,NE,0,0,2543773,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"2 DL, 3 LB, 6 DB",left,2017-09-08 00:44:06,2017-09-08 00:44:05,8,190.5,111.130131,1982-11-22,Texas,ILB,NE,KC,1,Gillette Stadium,"Foxborough, MA",outdoor,Field Turf,limpo,17.222222,77.0,8,south west,22,11,1982,44,5,44,6,1.0,False,0,1,0,91.440000,1600.20000,3.0
4,4,2017090700,20170907000118,away,69.32,35.42,1.82,2.43,0.16,12.63,164.31,2530794,Ron Parker,38,2017,35,1,854.0,NE,3,2,NE,0,0,2543773,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"2 DL, 3 LB, 6 DB",left,2017-09-08 00:44:06,2017-09-08 00:44:05,8,182.9,93.440028,1987-08-17,Newberry,FS,NE,KC,1,Gillette Stadium,"Foxborough, MA",outdoor,Field Turf,limpo,17.222222,77.0,8,south west,17,8,1987,44,5,44,6,1.0,False,0,1,0,11.430000,200.02500,3.0


In [41]:
# Funcao para realizar feature engineering no dataset treino
def features_estatisticas(data):

    df = pd.DataFrame()
    
    for col in data.columns:
        if col in ['index', 'GameId', 'PlayId', 'Team','IsRusher', 'PossessionTeam','FieldPosition', 'OffenseFormation', 'OffensePersonnel', 
                   'DefensePersonnel', 'PlayDirection', 'PlayerCollegeName', 'Position',
                   'HomeTeamAbbr', 'VisitorTeamAbbr', 'Stadium', 'Location', 'StadiumType', 'NflIdRusher',
                   'Turf', 'GameWeather', 'WindDirection', 'IsRusher', 'DisplayName', 'NflId', 'JerseyNumber',
                   'Season', 'Yards', 'PlayerBirthDate', 'TimeHandoff', 'TimeSnap']:
            continue
        print(col)  
        df[col + '_mean'] = data.groupby(['PlayId'])[col].mean()
        df[col + '_median'] = data.groupby(['PlayId'])[col].median()
        df[col + '_max'] = data.groupby(['PlayId'])[col].max()
        df[col + '_min'] = data.groupby(['PlayId'])[col].min()
        df[col + '_std'] = data.groupby(['PlayId'])[col].std()
        df[col + '_range'] = df[col + '_max'] - df[col + '_min']
        df[col + '_maxtoMin'] = df[col + '_max'] / df[col + '_min']
        df[col + '_mean_abs_chg'] = data.groupby(['PlayId'])[col].apply(lambda x: np.mean(np.abs(np.diff(x))))
        df[col + '_mean_change_of_abs_change'] = data.groupby(['PlayId'])[col].apply(mean_change_of_abs_change)
        df[col + '_abs_max'] = data.groupby(['PlayId'])[col].apply(lambda x: np.max(np.abs(x)))
        df[col + '_abs_min'] = data.groupby(['PlayId'])[col].apply(lambda x: np.min(np.abs(x)))
        df[col + '_abs_avg'] = (df[col + '_abs_min'] + df[col + '_abs_max'])/2

        # Advanced Features
        df[col + '_skew'] = data.groupby(['PlayId'])[col].skew()
        df[col + '_mad'] = data.groupby(['PlayId'])[col].mad()
        df[col + '_q25'] = data.groupby(['PlayId'])[col].quantile(0.25)
        df[col + '_q75'] = data.groupby(['PlayId'])[col].quantile(0.75)
        df[col + '_q95'] = data.groupby(['PlayId'])[col].quantile(0.95)
        df[col + '_iqr'] = df[col + '_q75'] - df[col + '_q25']
        df[col + '_SSC'] = data.groupby(['PlayId'])[col].apply(SSC) 
        df[col + '_skewness'] = data.groupby(['PlayId'])[col].apply(skewness)
        df[col + '_wave_lenght'] = data.groupby(['PlayId'])[col].apply(wave_length)
        df[col + '_norm_entropy'] = data.groupby(['PlayId'])[col].apply(norm_entropy)
        df[col + '_SRAV'] = data.groupby(['PlayId'])[col].apply(SRAV)
        df[col + '_kurtosis'] = data.groupby(['PlayId'])[col].apply(_kurtosis) 
        df[col + '_zero_crossing'] = data.groupby(['PlayId'])[col].apply(zero_crossing) 

    return df
    

In [44]:
# Criando um novo dataset aplicando Feature Engineering
data = features_estatisticas(train_df)

X
Y
S
A
Dis
Orientation
Dir
YardLine
Quarter
GameClock
Down
Distance
HomeScoreBeforePlay
VisitorScoreBeforePlay
DefendersInTheBox
PlayerHeight
PlayerWeight
Week
Temperature
Humidity
WindSpeed
PlayerBirthDate_day
PlayerBirthDate_month
PlayerBirthDate_year
TimeSnap_min
TimeSnap_seg
TimeHandoff_min
TimeHandoff_seg
TimeDifer
Morning
Afternoon
Evening
seconds_need_to_first_down
seconds_need_to_YardsLine
DefendersInTheBox_vs_Distance


In [47]:
data = data.reset_index()

In [53]:
# Executar somente para alguns modelos que nao tratam valores INF ou NAN
data.replace(-np.inf,0,inplace=True)
data.replace(np.inf,0,inplace=True)
data.fillna(0, inplace=True)

In [54]:
data.shape

(23171, 876)

In [55]:
data.head()

,PlayId,X_mean,X_median,X_max,X_min,X_std,X_range,X_maxtoMin,X_mean_abs_chg,X_mean_change_of_abs_change,X_abs_max,X_abs_min,X_abs_avg,X_skew,X_mad,X_q25,X_q75,X_q95,X_iqr,X_SSC,X_skewness,X_wave_lenght,X_norm_entropy,X_SRAV,X_kurtosis,X_zero_crossing,Y_mean,Y_median,Y_max,Y_min,Y_std,Y_range,Y_maxtoMin,Y_mean_abs_chg,Y_mean_change_of_abs_change,Y_abs_max,Y_abs_min,Y_abs_avg,Y_skew,Y_mad,Y_q25,Y_q75,Y_q95,Y_iqr,Y_SSC,Y_skewness,Y_wave_lenght,Y_norm_entropy,Y_SRAV,Y_kurtosis,Y_zero_crossing,S_mean,S_median,S_max,S_min,S_std,S_range,S_maxtoMin,S_mean_abs_chg,S_mean_change_of_abs_change,S_abs_max,S_abs_min,S_abs_avg,S_skew,S_mad,S_q25,S_q75,S_q95,S_iqr,S_SSC,S_skewness,S_wave_lenght,S_norm_entropy,S_SRAV,S_kurtosis,S_zero_crossing,A_mean,A_median,A_max,A_min,A_std,A_range,A_maxtoMin,A_mean_abs_chg,A_mean_change_of_abs_change,A_abs_max,A_abs_min,A_abs_avg,A_skew,A_mad,A_q25,A_q75,A_q95,A_iqr,A_SSC,A_skewness,A_wave_lenght,A_norm_entropy,A_SRAV,A_kurtosis,A_zero_crossing,Dis_mean,Dis_median,Dis_max,Dis_min,Dis_std,Dis_range,Dis_maxtoMin,Dis_mean_abs_chg,Dis_mean_change_of_abs_change,Dis_abs_max,Dis_abs_min,Dis_abs_avg,Dis_skew,Dis_mad,Dis_q25,Dis_q75,Dis_q95,Dis_iqr,Dis_SSC,Dis_skewness,Dis_wave_lenght,Dis_norm_entropy,Dis_SRAV,Dis_kurtosis,Dis_zero_crossing,Orientation_mean,Orientation_median,Orientation_max,Orientation_min,Orientation_std,Orientation_range,Orientation_maxtoMin,Orientation_mean_abs_chg,Orientation_mean_change_of_abs_change,Orientation_abs_max,Orientation_abs_min,Orientation_abs_avg,Orientation_skew,Orientation_mad,Orientation_q25,Orientation_q75,Orientation_q95,Orientation_iqr,Orientation_SSC,Orientation_skewness,Orientation_wave_lenght,Orientation_norm_entropy,Orientation_SRAV,Orientation_kurtosis,Orientation_zero_crossing,Dir_mean,Dir_median,Dir_max,Dir_min,Dir_std,Dir_range,Dir_maxtoMin,Dir_mean_abs_chg,Dir_mean_change_of_abs_change,Dir_abs_max,Dir_abs_min,Dir_abs_avg,Dir_skew,Dir_mad,Dir_q25,Dir_q75,Dir_q95,Dir_iqr,Dir_SSC,Dir_skewness,Dir_wave_lenght,Dir_norm_entropy,Dir_SRAV,Dir_kurtosis,Dir_zero_crossing,YardLine_mean,YardLine_median,YardLine_max,YardLine_min,YardLine_std,YardLine_range,YardLine_maxtoMin,YardLine_mean_abs_chg,YardLine_mean_change_of_abs_change,YardLine_abs_max,YardLine_abs_min,YardLine_abs_avg,YardLine_skew,YardLine_mad,YardLine_q25,YardLine_q75,YardLine_q95,YardLine_iqr,YardLine_SSC,YardLine_skewness,YardLine_wave_lenght,YardLine_norm_entropy,YardLine_SRAV,YardLine_kurtosis,YardLine_zero_crossing,Quarter_mean,Quarter_median,Quarter_max,Quarter_min,Quarter_std,Quarter_range,Quarter_maxtoMin,Quarter_mean_abs_chg,Quarter_mean_change_of_abs_change,Quarter_abs_max,Quarter_abs_min,Quarter_abs_avg,Quarter_skew,Quarter_mad,Quarter_q25,Quarter_q75,Quarter_q95,Quarter_iqr,Quarter_SSC,Quarter_skewness,Quarter_wave_lenght,Quarter_norm_entropy,Quarter_SRAV,Quarter_kurtosis,Quarter_zero_crossing,GameClock_mean,GameClock_median,GameClock_max,GameClock_min,GameClock_std,GameClock_range,GameClock_maxtoMin,GameClock_mean_abs_chg,GameClock_mean_change_of_abs_change,GameClock_abs_max,GameClock_abs_min,GameClock_abs_avg,GameClock_skew,GameClock_mad,GameClock_q25,GameClock_q75,GameClock_q95,GameClock_iqr,GameClock_SSC,GameClock_skewness,GameClock_wave_lenght,GameClock_norm_entropy,GameClock_SRAV,GameClock_kurtosis,GameClock_zero_crossing,Down_mean,Down_median,Down_max,Down_min,Down_std,Down_range,Down_maxtoMin,Down_mean_abs_chg,Down_mean_change_of_abs_change,Down_abs_max,Down_abs_min,Down_abs_avg,Down_skew,Down_mad,Down_q25,Down_q75,Down_q95,Down_iqr,Down_SSC,Down_skewness,Down_wave_lenght,Down_norm_entropy,Down_SRAV,Down_kurtosis,Down_zero_crossing,Distance_mean,Distance_median,Distance_max,Distance_min,Distance_std,Distance_range,Distance_maxtoMin,Distance_mean_abs_chg,Distance_mean_change_of_abs_change,Distance_abs_max,Distance_abs_min,Distance_abs_avg,Distance_skew,Distance_mad,Distance_q25,Distance_q75,Distance_q95,Distance_iqr,Distance_SSC,Distance_skewness,Distance_wave_lenght,Distance_norm_entropy,Distan

In [56]:
target = train_df.groupby(['PlayId'])['Yards'].mean()
target = target.reset_index()

In [57]:
train_df = pd.merge(data, target, on='PlayId', how='outer')

In [58]:
train_df.head()

,PlayId,X_mean,X_median,X_max,X_min,X_std,X_range,X_maxtoMin,X_mean_abs_chg,X_mean_change_of_abs_change,X_abs_max,X_abs_min,X_abs_avg,X_skew,X_mad,X_q25,X_q75,X_q95,X_iqr,X_SSC,X_skewness,X_wave_lenght,X_norm_entropy,X_SRAV,X_kurtosis,X_zero_crossing,Y_mean,Y_median,Y_max,Y_min,Y_std,Y_range,Y_maxtoMin,Y_mean_abs_chg,Y_mean_change_of_abs_change,Y_abs_max,Y_abs_min,Y_abs_avg,Y_skew,Y_mad,Y_q25,Y_q75,Y_q95,Y_iqr,Y_SSC,Y_skewness,Y_wave_lenght,Y_norm_entropy,Y_SRAV,Y_kurtosis,Y_zero_crossing,S_mean,S_median,S_max,S_min,S_std,S_range,S_maxtoMin,S_mean_abs_chg,S_mean_change_of_abs_change,S_abs_max,S_abs_min,S_abs_avg,S_skew,S_mad,S_q25,S_q75,S_q95,S_iqr,S_SSC,S_skewness,S_wave_lenght,S_norm_entropy,S_SRAV,S_kurtosis,S_zero_crossing,A_mean,A_median,A_max,A_min,A_std,A_range,A_maxtoMin,A_mean_abs_chg,A_mean_change_of_abs_change,A_abs_max,A_abs_min,A_abs_avg,A_skew,A_mad,A_q25,A_q75,A_q95,A_iqr,A_SSC,A_skewness,A_wave_lenght,A_norm_entropy,A_SRAV,A_kurtosis,A_zero_crossing,Dis_mean,Dis_median,Dis_max,Dis_min,Dis_std,Dis_range,Dis_maxtoMin,Dis_mean_abs_chg,Dis_mean_change_of_abs_change,Dis_abs_max,Dis_abs_min,Dis_abs_avg,Dis_skew,Dis_mad,Dis_q25,Dis_q75,Dis_q95,Dis_iqr,Dis_SSC,Dis_skewness,Dis_wave_lenght,Dis_norm_entropy,Dis_SRAV,Dis_kurtosis,Dis_zero_crossing,Orientation_mean,Orientation_median,Orientation_max,Orientation_min,Orientation_std,Orientation_range,Orientation_maxtoMin,Orientation_mean_abs_chg,Orientation_mean_change_of_abs_change,Orientation_abs_max,Orientation_abs_min,Orientation_abs_avg,Orientation_skew,Orientation_mad,Orientation_q25,Orientation_q75,Orientation_q95,Orientation_iqr,Orientation_SSC,Orientation_skewness,Orientation_wave_lenght,Orientation_norm_entropy,Orientation_SRAV,Orientation_kurtosis,Orientation_zero_crossing,Dir_mean,Dir_median,Dir_max,Dir_min,Dir_std,Dir_range,Dir_maxtoMin,Dir_mean_abs_chg,Dir_mean_change_of_abs_change,Dir_abs_max,Dir_abs_min,Dir_abs_avg,Dir_skew,Dir_mad,Dir_q25,Dir_q75,Dir_q95,Dir_iqr,Dir_SSC,Dir_skewness,Dir_wave_lenght,Dir_norm_entropy,Dir_SRAV,Dir_kurtosis,Dir_zero_crossing,YardLine_mean,YardLine_median,YardLine_max,YardLine_min,YardLine_std,YardLine_range,YardLine_maxtoMin,YardLine_mean_abs_chg,YardLine_mean_change_of_abs_change,YardLine_abs_max,YardLine_abs_min,YardLine_abs_avg,YardLine_skew,YardLine_mad,YardLine_q25,YardLine_q75,YardLine_q95,YardLine_iqr,YardLine_SSC,YardLine_skewness,YardLine_wave_lenght,YardLine_norm_entropy,YardLine_SRAV,YardLine_kurtosis,YardLine_zero_crossing,Quarter_mean,Quarter_median,Quarter_max,Quarter_min,Quarter_std,Quarter_range,Quarter_maxtoMin,Quarter_mean_abs_chg,Quarter_mean_change_of_abs_change,Quarter_abs_max,Quarter_abs_min,Quarter_abs_avg,Quarter_skew,Quarter_mad,Quarter_q25,Quarter_q75,Quarter_q95,Quarter_iqr,Quarter_SSC,Quarter_skewness,Quarter_wave_lenght,Quarter_norm_entropy,Quarter_SRAV,Quarter_kurtosis,Quarter_zero_crossing,GameClock_mean,GameClock_median,GameClock_max,GameClock_min,GameClock_std,GameClock_range,GameClock_maxtoMin,GameClock_mean_abs_chg,GameClock_mean_change_of_abs_change,GameClock_abs_max,GameClock_abs_min,GameClock_abs_avg,GameClock_skew,GameClock_mad,GameClock_q25,GameClock_q75,GameClock_q95,GameClock_iqr,GameClock_SSC,GameClock_skewness,GameClock_wave_lenght,GameClock_norm_entropy,GameClock_SRAV,GameClock_kurtosis,GameClock_zero_crossing,Down_mean,Down_median,Down_max,Down_min,Down_std,Down_range,Down_maxtoMin,Down_mean_abs_chg,Down_mean_change_of_abs_change,Down_abs_max,Down_abs_min,Down_abs_avg,Down_skew,Down_mad,Down_q25,Down_q75,Down_q95,Down_iqr,Down_SSC,Down_skewness,Down_wave_lenght,Down_norm_entropy,Down_SRAV,Down_kurtosis,Down_zero_crossing,Distance_mean,Distance_median,Distance_max,Distance_min,Distance_std,Distance_range,Distance_maxtoMin,Distance_mean_abs_chg,Distance_mean_change_of_abs_change,Distance_abs_max,Distance_abs_min,Distance_abs_avg,Distance_skew,Distance_mad,Distance_q25,Distance_q75,Distance_q95,Distance_iqr,Distance_SSC,Distance_skewness,Distance_wave_lenght,Distance_norm_entropy,Distan

In [59]:
data.to_csv('../data/data.csv', index=False)
target.to_csv('../data/target.csv', index=False)
train_df.to_csv('../data/train_df.csv', index=False)

# Criação e Validação dos Modelos de ML

In [60]:
# Fazendo uma limpeza na memoria
gc.collect()

375

In [61]:
# Setup cross validation folds
kf = 5
folds = KFold(n_splits=kf, shuffle=False, random_state=42)
print(str(kf) + ' Folds para treino...')

5 Folds para treino...


In [62]:
# Identificando as features para o modelo
features = list(train_df.columns)
X = train_df[features]

In [63]:
# Split features and labels
X_train = train_df.drop(['Yards', 'PlayId'],axis=1)
y_train = train_df['Yards']

# Aplicando a mesma escala nos dados
scaler = MinMaxScaler() 
X_train = scaler.fit_transform(X_train) 

In [64]:
best_params_lgb = {
    "boosting": "gbdt",
    "verbosity": -1,
    "num_leaves":3,
    "min_data_in_leaf": 10,
    "max_depth": -1,
    "learning_rate": 0.0005,
    "bagging_freq": 4,
    "bagging_fraction": 0.1,
    "bagging_seed": 11,
    "feature_fraction" : 1,
    "random_seed": 19,
    "metric": "rmse",
    "boost_from_average" : False
}

In [69]:
oof = np.zeros(len(X_train))
tr_rmse  = []
val_rmse = []
models   = []

for fold_, (trn_idx, val_idx) in enumerate(folds.split(X_train,y_train)):
    strLog = "fold {}".format(fold_)
    print(strLog)
    
    X_tr, X_val = train_df.iloc[trn_idx][features], train_df.iloc[val_idx][features]
    train_y, y_val = y_train[trn_idx], y_train[val_idx]

    model = lgb.LGBMRegressor(**best_params_lgb, n_estimators = 100, n_jobs = -1)
    model.fit(X_tr, 
              train_y, 
              eval_set=[(X_tr, train_y), (X_val, y_val)], 
              eval_metric='rsme',
              verbose=10, 
              early_stopping_rounds=99)
    
    oof[val_idx] = model.predict(X_val)
    
    val_score = mean_squared_error(y_val, oof[val_idx])
    val_rmse.append(val_score)
    
    tr_score = mean_squared_error(train_y, model.predict(X_tr))
    tr_rmse.append(tr_score)
    
    models.append(model)

fold 0
Training until validation scores don't improve for 99 rounds
[10]	training's rmse: 7.64526	valid_1's rmse: 7.72931
[20]	training's rmse: 7.61557	valid_1's rmse: 7.7006
[30]	training's rmse: 7.58502	valid_1's rmse: 7.66996
[40]	training's rmse: 7.5549	valid_1's rmse: 7.64097
[50]	training's rmse: 7.5262	valid_1's rmse: 7.61383
[60]	training's rmse: 7.49711	valid_1's rmse: 7.585
[70]	training's rmse: 7.46841	valid_1's rmse: 7.55747
[80]	training's rmse: 7.44	valid_1's rmse: 7.52932
[90]	training's rmse: 7.41253	valid_1's rmse: 7.50332
[100]	training's rmse: 7.38469	valid_1's rmse: 7.47611
Did not meet early stopping. Best iteration is:
[100]	training's rmse: 7.38469	valid_1's rmse: 7.47611
fold 1
Training until validation scores don't improve for 99 rounds
[10]	training's rmse: 7.73186	valid_1's rmse: 7.36841
[20]	training's rmse: 7.70245	valid_1's rmse: 7.34042
[30]	training's rmse: 7.67147	valid_1's rmse: 7.3107
[40]	training's rmse: 7.6418	valid_1's rmse: 7.28263
[50]	training'

# Realizando a submissão

In [ ]:
# Funcao para realizar feature engineering no dataset de teste
def feature_engineering_test(df): 
    
    # Limpeza e conversao dos dados
    fill_na(df)
    convert_data(df)
    
    df_median = df.median()
    
    # Conversao de algumas features
    df['PlayerHeight']  = df['PlayerHeight'].apply(convert_to_cm)
    df['PlayerWeight']  = df['PlayerWeight'].apply(convert_to_kg)
    df['Temperature']   = df['Temperature'].apply(convert_to_celsius)
    df['StadiumType']   = df['StadiumType'].apply(agrupar_tipo_estadio)
    df['Stadium']       = df['Stadium'].apply(agrupar_estadio)
    df['Location']      = df['Location'].apply(agrupar_local)
    df['Turf']          = df['Turf'].apply(agrupar_gramado)
    df['WindDirection'] = df['WindDirection'].apply(agrupa_wind_direction)
    df['WindSpeed']     = df['WindSpeed'].apply(convert_wind_speed)
    df['GameWeather']   = df['GameWeather'].apply(agrupar_clima)


    # Corrigindo a feature Stadium
    df.loc[df['Stadium'] == 'MetLife Stadium', 'StadiumType'] = 'outdoor'
    df.loc[df['Stadium'] == 'StubHub Center', 'StadiumType'] = 'outdoor'    
    
    # Nova feature com a diferença entre o tempo de lançamento da bola até quando o jogador captura
    df['TimeDifer'] = df.apply(lambda row: (row['TimeHandoff'] - row['TimeSnap']).total_seconds(), axis=1)            
         
    # Nova feature para indicar se é o jogador que esta realizando a jogada (corredor)
    df['IsRusher'] = df['NflId'] == df['NflIdRusher']
    
    # Novas features com base no horario do jogo
    df['Morning']   = df['GameClock'].apply(lambda x : 1 if (int(x[0:2]) >=0 and int(x[0:2]) <12) else 0)
    df['Afternoon'] = df['GameClock'].apply(lambda x : 1 if (int(x[0:2]) <18 and int(x[0:2]) >=12) else 0)
    df['Evening']   = df['GameClock'].apply(lambda x : 1 if (int(x[0:2]) >= 18 and int(x[0:2]) < 24) else 0)
    df['GameClock'] = df['GameClock'].apply(str_to_seconds) 
    
    # Criando novas features com dados de Distance, YardLine e DefendersInTheBox
    df['seconds_need_to_first_down'] = (df['Distance']*0.9144)/df['Dis']
    df['seconds_need_to_YardsLine'] = (df['YardLine']*0.9144)/df['Dis']    
    df['DefendersInTheBox_vs_Distance'] = df['DefendersInTheBox'] / df['Distance']
    
    # Ordenacao do dataset e renovando o index
    df = df.sort_values(by=['PlayId', 'Team', 'IsRusher']).reset_index()
    
    # Removendo colunas que não serão utilizadas
    df = df.drop(['index', 'GameId','PlayId','NflId', 'DisplayName','NflIdRusher', 'TimeHandoff', 'TimeSnap', 'PlayerBirthDate'], axis=1)

    # Atribuindo media para os demais dados missing
    df.fillna(df_median, inplace=True)

    # Executar somente para alguns modelos que nao tratam valores INF ou NAN
    #train_df.replace(-np.inf,0,inplace=True)
    #train_df.replace(np.inf,0,inplace=True)

    # Removendo todas as variaveis categoricas
    cat_features = []
    for col in df.columns:
        if df[col].dtype =='object':
            cat_features.append(col)

    df = df.drop(cat_features, axis=1)
    
    return df

In [ ]:
from kaggle.competitions import nflrush

pd.options.mode.chained_assignment = None
index = 0

env = nflrush.make_env()

for (test, sample_prediction_df) in tqdm.tqdm(env.iter_test()):
    df_test = feature_engineering_test(test)
    
    count=0
    test_data = np.zeros((1,len(features)))

    for c in features:
        if c in df_test:
            try:
                test_data[0][count] = df_test[c][index]
            except:
                test_data[0][count] = np.nan
            count+=1
    
    y_pred = np.zeros(199)        
    y_pred_p = np.sum(np.round(scaler.inverse_transform([model.predict(test_data)[0] for model in models])))/kf
    y_pred_p += 99
    
    for j in range(199):
        if j>=y_pred_p+10:
            y_pred[j]=1.0
        elif j>=y_pred_p-10:
            y_pred[j]=(j+10-y_pred_p)*0.05
    
    env.predict(pd.DataFrame(data=[y_pred],columns=sample_prediction_df.columns))
    
    index += 22
    
env.write_submission_file()

print([filename for filename in os.listdir('/kaggle/working') if '.csv' in filename])